# Survival prediction of titanic passengers

---
## Set up 

### Load modules

In [ ]:
# data handling
import numpy as np
import pandas as pd

# visualisation
from matplotlib import pyplot as plt

# preprocessing
from sklearn.preprocessing import OrdinalEncoder

# classification algorithms
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

# ML tools
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

#local modules
from barplot import *

### Set display options

In [ ]:
# allow multiple outputs per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Plot the Figures Inline
%matplotlib inline

# Prevent label cut off from figures 
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

---
## Data loader

In [ ]:
# get metadata
meta_data = pd.read_csv("data/metadata.csv")
meta_data

In [ ]:
# load train data
train_data = pd.read_csv("data/titanic-train.csv")
print("Shape: ", train_data.shape)
train_data.head()

In [ ]:
# load test data
test_data = pd.read_csv("data/titanic-test.csv")
print("Shape: ", test_data.shape)
test_data.head()

---
## Data exploration

### Check the frequency of classes in the predicted variable (label) in the training set

In [ ]:
num_deceased = (train_data["Survived"] == 0).sum()
num_survived = (train_data["Survived"] == 1).sum()
assert num_deceased + num_survived == 891
print("deceased total: ", num_deceased, " - deceased %: ", round(100/891*num_deceased, 1))
print("survived total: ", num_survived, " - deceased %: ", round(100/891*num_survived, 1))

Conclusion: the number of fatalities is much higher than the number of survivors. This means that the dataset is imbalanced.

### Check if the datasets contain missing values

In [ ]:
missing_values = pd.DataFrame({'Training set': train_data.isna().sum(), 
                               'Test set': test_data.isna().sum()})
missing_values

Conclusion: There are many missing values for the age of passengers and the cabin type. Therefore, these features will be excluded from the following analyses.

### Count the number of unique values of features of interest

In [ ]:
train_data["Sex"].nunique()
train_data["SibSp"].nunique()
train_data["Parch"].nunique()
train_data["Fare"].nunique()

Conclusion: there are many different fares that are assumably associated with the ticket class. Let's check this: 

### Investigate fares

In [ ]:
# check min and max prices of fares per class

# divide training dataset per class
class1 = train_data.loc[train_data['Pclass'] == 1]
class2 = train_data.loc[train_data['Pclass'] == 2]
class3 = train_data.loc[train_data['Pclass'] == 3]
# save classes in list
classes = [class1, class2, class3]

# print fare ranges
for i, pclass in enumerate(classes):    
    print(f"Max fare class {i+1}: ", pclass["Fare"].max())
    print(f"Min fare class {i+1}: ",pclass["Fare"].min())
    print()

In [ ]:
# plot fares per class as histograms

# save fares in numpy array
fares_per_class = [class1["Fare"].to_numpy(), 
             class2["Fare"].to_numpy(), 
             class3["Fare"].to_numpy()]

# plot fares
fig, ax = plt.subplots(1,len(fares_per_class), figsize=(15, 5))
for i, data in enumerate(fares_per_class):
    _ = ax[i].hist(data, bins=20)
    _ = ax[i].set_title(f"Class {i+1}")
    _ = ax[i].set_xlabel("Fares")
    _ = ax[i].set_ylabel("Frequency")

Conclusion: The fares of the 3 different classes overlap, especially the fares of class 2 and 3. It might therefore be more useful to predict survival rates depending on passenger class rather than fare. Let's check among the categorical features if there are categories that are (strongly) associated with survival rate.

### Investigate survival rates per categories

#### Passenger class: 

In [ ]:
# save categories in list and convert them to string variables for plotting
categories_class = list(map(str, train_data["Pclass"].unique()))
categories_class.sort()
categories_class # check result

# calculate percentage of survivors per passenger class
survivors_per_class = []
for pclass in classes: 
    surv = round(pclass["Survived"].sum()/len(pclass["Survived"])*100, 1)
    survivors_per_class.append(surv)

In [ ]:
# plot survivors per class
plot_survivors_per_category(categories_class, 
             survivors_per_class, 
             title="Survivors per passenger class", 
             xlabel="Passenger classes")

In [ ]:
# compare result to total survivors per passenger class
train_data["Pclass"].value_counts()

Conclusion: the survival rate seems to be correlated to the passenger class and therefore likely influences the prediction of survival. Although the survival rate is highest for passengers in class 1, most people travelled in class 3.

#### Gender:

In [ ]:
# save categories in list
categories_gender = list(map(str, train_data["Sex"].unique()))
categories_gender # check result

# calculate percentage of survivors per gender
men = train_data.loc[train_data.Sex == 'male']["Survived"].to_numpy()
women = train_data.loc[train_data.Sex == 'female']["Survived"].to_numpy()
men_surv = round(sum(men)/len(men)*100, 1)
women_surv = round(sum(women)/len(women)*100, 1)

# store results in list
survivors_per_gender = [men_surv, women_surv]

In [ ]:
# plot survivors per gender
plot_survivors_per_category(categories_gender, 
             survivors_per_gender, 
             title="Survivors per gender", 
             xlabel="Gender")

In [ ]:
# compare result to total survivors per gender
train_data["Sex"].value_counts()

Conclusion: the survival rate of women is much higher than the survival rate of men although the number of men aboard the Titanic was much higher compared to the number of women. Therefore, the gender likely has a strong influence on the prediction of survival.

#### Number of siblings/ spouses aboard

In [ ]:
# save categories in list
categories_sibsp = list(train_data["SibSp"].unique())
categories_sibsp.sort()

# calculate percentage of survivors per number of siblings/ spouses aboard
# and save results in list
survivors_per_sibsp = []
for i in categories_sibsp:
    sibsp = train_data.loc[train_data.SibSp == i]["Survived"].to_numpy()
    survivors_per_sibsp.append(round(sum(sibsp)/len(sibsp)*100, 1))

# convert categories to string variables for plotting
categories_sibsp = list(map(str, categories_sibsp))
categories_sibsp # check result

In [ ]:
# plot survivors per number of siblings/ spouses aboard
plot_survivors_per_category(categories_sibsp, 
             survivors_per_sibsp, 
             title="Survivors per number of siblings/ spouses aboard", 
             xlabel="Number of siblings/ spouses aboard")

In [ ]:
# compare result to total survivors per siblings/ spouses aboard
train_data["SibSp"].value_counts()

Conclusion: The people with 1 or 2 siblings/ spouses aboard had the highest rate of survival. This could mean that these people had support from family members with getting a spot in one of the lifeboats. However, the number of people who travelled with no siblings or spouses is more than twice as high as the number of people who travelled in company. Therefore, the number of siblings/ spouses might be weakly associated with the chance of survival.

#### Number of parents/ children aboard

In [ ]:
# save categories in list
categories_parch = list(train_data["Parch"].unique())
categories_parch.sort()

# calculate percentage of survivors per number of parents/ children aboard
# and save results in list
survivors_per_parch = []
for i in categories_parch:
    parch = train_data.loc[train_data.Parch == i]["Survived"].to_numpy()
    survivors_per_parch.append(round(sum(parch)/len(parch)*100, 1))

# convert categories to string variables for plotting
categories_parch = list(map(str, categories_parch))
categories_parch # check result

In [ ]:
# plot survivors per number of parents/ children aboard
plot_survivors_per_category(categories_parch, 
             survivors_per_parch, 
             title="Survivors per number of parents/ children aboard", 
             xlabel="Number of parents/ children aboard")

In [ ]:
# compare result to total survivors per parents/ children aboard
train_data["Parch"].value_counts()

Conclusion: The people who had between 1 and 3 parents/ children aboard had the highest rate of survival. As above, this could mean that these people had support from family members with getting a spot in one of the lifeboats. However, most people travelled without company. Therefore, the number of parents/ children aboard might be weakly associated with the chance of survival.

#### Port of embarkation

In [ ]:
# save categories in list
categories_embarked = list(map(str, train_data["Embarked"].unique()))
categories_embarked

# calculate percentage of survivors per port of embarkation
# note: leave out the two passengers of unknown port of embarkation
survivors_per_port = []
for i in categories_embarked[:3]:
    port = train_data.loc[train_data.Embarked == i]["Survived"].to_numpy()
    survivors_per_port.append(round(sum(port)/len(port)*100, 1))

In [ ]:
# plot survivors per port of embarkation
plot_survivors_per_category(categories_embarked[:3], 
             survivors_per_port, 
             title="Survivors per port of embarkation", 
             xlabel="Port of embarkation")

In [ ]:
# compare result to total survivors per port of embarkation
train_data["Embarked"].value_counts()

Conclusion: Although by far the most people embarked in Southampton the percentage of survivors who embarked in Cherbourg is higher compared to Southampton and Queenstown. This could be due to many first class passengers having embarked here. Let's check this: 

#### Passengers per class per port

In [ ]:
# calculate number of survivors per class and port of embarkation
# note: leave out the two passengers of unknown port of embarkation

survivors_class_port = []
# loop over classes
for pclass in classes: 
    survivors_per_port_pclass = []
    # loop over ports
    for cat in categories_embarked[:3]:
        port = pclass.loc[pclass.Embarked == cat]["Survived"].to_numpy().sum()
        survivors_per_port_pclass.append(port)
    survivors_class_port.append(survivors_per_port_pclass)

survivors_class_port

In [ ]:
# plot survivors per class per port of embarkation

# set variables
x = np.arange(len(categories_embarked[:3]))  # the label locations
width = 0.2  # the width of the bars

# set up plot
fig, ax = plt.subplots()
_ = ax.set_title("Survivors per class per port")
_ = ax.set_ylabel("Survivors (toal)")
_ = ax.set_xticks(x)
_ = ax.set_xticklabels(categories_embarked[:3])

# plot barplot
for i,j in zip(survivors_class_port,range(-1,2)): 
    _ = ax.bar(x=x+width*j, height=i, width=width, label=f'Class {j+2}')
    # annotate barplot
    for k, data in enumerate(i):
        _ = ax.annotate(s=data, xy=(k+width*j, data+0.7), ha='center')
_ = ax.legend()

Conclusion: Most survivors, irrespective of class, embarked in Southampton. However, in Cherbourg a higher number of survivors belonging to the first class embarked compared to second and thrid class survivors. Additionally, in Queenstown a higher number of survivors belonging to the third class embarked compared to first and second class survivors. Therefore, the port of embarkation might have a weak influence on the prediction of survival.

### Summary

Based on this data exploration, the features that likely influence the prediction of survival are in presumed descending order of strength: 
- gender
- passenger class
- siblings/ spouses aboard; children/ parents aboard; port of embarkation/ fare

---
## Data preparation

### Select features to be included in the models

In [ ]:
# select features and label column
features = ["Sex", "Pclass", "SibSp", "Parch", "Embarked", "Survived"] #

# create pruned dataset
train_data_pruned = train_data[features]

In [ ]:
# drop rows with missing values and check result
X_pruned = train_data_pruned.dropna()
X_pruned.shape
X_pruned.head()

In [ ]:
# store labels in separate variable
y = X_pruned['Survived']
X = X_pruned.drop(columns=['Survived'])

### Encode categorical features as ordinal integers

In [ ]:
enc = OrdinalEncoder()
enc.fit(X)
X_ord = enc.transform(X)

---
## Models

We now train different classic machine learning models on the training set and evaluate their performance with k-fold cross validation.

### Run different classic machine learning models: 
- Random Forest
- Gradient Boosting
- Logistic Regression
- Suport Vector Machine

In [ ]:
# define models
rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0, n_jobs=-1, class_weight={0:2, 1:1})
gb = GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=0)
lr = LogisticRegression(random_state=0, solver='liblinear', multi_class='ovr', max_iter=100, class_weight={0:2, 1:1})
svm = LinearSVC(random_state=0, C=1.0, max_iter=1000, class_weight={0:2, 1:1})

# save models in list
models = [rf, gb, lr, svm]
# save model names in list
model_names = ['RF', 'GB', 'LR', 'SVM']

In [ ]:
%%time
# perform cross validation
accuracies = []
std = []
for model, names in zip(models, model_names):
    scores = cross_val_score(model, X_ord, y, cv=5)
    accuracies.append(scores.mean())
    std.append(scores.std())
    print(f"Cross val scores {names}: ", np.around(scores, decimals=2))
    print(f"Mean and stdev: {scores.mean():.2f} +/- {scores.std():.2f}")
    print()

### Visualise results

In [ ]:
# plot accuracies of different ML models
plot_model_accuracies(model_names, accuracies, std)

Conclusion: the performances of the different classifiers are in a narrow range with mean accuracies between 78% and 80%, and standard deviations between 1% and 3%. In order to better understand the models, let's have a look into model explainability by investigating the importances of individual features.

### Check feature importances
#### Calculate the impurity-based feature importances of the tree classifiers

In [ ]:
# random forest classifier
_ = rf.fit(X_ord, y)
importances_rf = rf.feature_importances_

In [ ]:
# gradient boosting classifier
_ = gb.fit(X_ord, y)
importances_gb = gb.feature_importances_

In [ ]:
# store values in list
importances_trees = [importances_rf.tolist(), importances_gb.tolist()]

#### Plot the impurity-based feature importances of the tree classifiers

In [ ]:
clf_names = ["Random forest", "Gradient boosting"] # classifier names

plot_feature_importance(clf=clf_names, 
                        feat=features, 
                        ylabel="Importance scores",
                        importances=importances_trees)

Conclusion: the feature importance scores are almost identical between the random forest classifier and the gradient boosting classifier. By far the most important feature is gender followed by passenger class. As postulated above, the features siblings/ spouses aboard, parents/ children aboard and port of embaraktion play a minor role in survival prediction. 

#### Calculate weights assigned to the features in the linear classifiers

In [ ]:
# logistic regression
_ = lr.fit(X_ord, y)
importances_lr = lr.coef_.ravel()*-1 # flatten array and make weights positive

In [ ]:
# support vector machine
_ = svm.fit(X_ord, y)
importances_svm = svm.coef_.ravel()*-1 # flatten array and make weights positive

In [ ]:
# store values in list
importances_linear = [importances_lr.tolist(), importances_svm.tolist()]

#### Plot weights assigned to the features in the linear classifiers

In [ ]:
clf_names = ["Logistic Regression", "Support Vector Machine"] # classifier names

plot_feature_importance(clf=clf_names, 
                        feat=features, 
                        ylabel="Feature weights",
                        importances=importances_linear)

Conclusion: overall, feature weights are higher in the logistic regression model compared to the support vector machine model. However, the relative importances between features within the models are roughly equal. As observed in the random forest model and the gradient boosting model, the most important feature is gender followed by passenger class while the features siblings/ spouses aboard, parents/ children aboard and port of embaraktion play a minor role in survival prediction. It is notable that the feature parents/ children aboard seems to be of less importance in the linear models compared to the tree classifiers. <br>
Note: the feature weights are negative but I turned them into positive numbers in order to make them more comparable to the importance scores of the tree classifiers. In binary classification negative feature weights indicate that a feature tends more towards predicting 0. Because the dataset is imbalanced towards fatalities (see data exploration part), feature weights are negative.

### Perform hyperparameter tuning

The hyperparameter settings for the models might not be optimal. Let's do a hyperparameter grid search to find out if the hyperparameter settings can be optimised.

In [ ]:
# define classifiers and store them in list
rf = RandomForestClassifier()
gb = GradientBoostingClassifier()
lr = LogisticRegression()
svm = LinearSVC()

classifiers = [rf, gb, lr, svm]

In [ ]:
# define parameters of classifiers and store them in list
parameters_rf = {'n_estimators':[50, 100, 200], 'max_depth':[None, 5, 10]}
parameters_gb = {'learning_rate':[0.05, 0.1, 0.5], 'n_estimators':[50, 100, 200], 'max_depth':[None, 5, 10]}
parameters_lr = {'solver':['newton-cg', 'lbfgs', 'liblinear'], 'max_iter':[100, 500, 1000]}
parameters_svm = {'max_iter':[1000, 3000, 5000]}

parameters = [parameters_rf, parameters_gb, parameters_lr, parameters_svm]

In [ ]:
# perform best parameter grid search per classifier and print results
for est, params, name in zip(classifiers, parameters, model_names): 
    clf = GridSearchCV(est, params, cv=5, n_jobs=-1)
    _ = clf.fit(X_ord, y)  
    
    # show results
    print("Model: ", name)
    print("Scores per params combination: ", np.around(clf.cv_results_['mean_test_score'], decimals=2))
    print("Best score: ", round(clf.best_score_, 2))
    print("Best parameter combination: ", clf.best_params_)
    print()

Conclusion: after performing hyperparameter tuning, the model accuracies are almost identical (± 1%) compared to the model accuracies previous to hyperparameter tuning. This means 1) that the hyperparameter settings that were initially chosen were likely (close to) optimal and 2) that all tested models show a certain robustness against changing hyperparameters. 